# Segment gaps between cells  
## 2.1 Option A: Applying an existing pixelclassifier 
We have trained a pixelclassifier (```edema_20241006```) to segment gaps between cells labelled as "fluid".  
Download the classifier from the ```pixel_classifiers``` folder in GitHub and place it your QuPath project folder under ```classifiers/pixel_classifiers/```  
Then **skip to 2.2**.

## 2.1. Option B: Training and applying a pixelclassifier from scratch  
#### 2.1.1 Sample training regions 
The following script samples non-overlapping square regions of interest within all regions classed as "tissue".  
Modify these variables as necessary:  
**```sidelength```** makes the boxes bigger or smaller  
**```n_regions```** changes the number of regions of interest to sample.  

Run the script [for all](https://qupath.readthedocs.io/en/0.4/docs/scripting/workflows_to_scripts.html#running-a-script-for-multiple-images) images you want to include in training.  

The regions are named "Other" to distinguish them from training annotations for the tissue training image.  


In [ ]:
import qupath.lib.roi.RectangleROI
import qupath.lib.objects.PathAnnotationObject
import qupath.lib.images.servers.ImageServer

double sidelength = 500 // side-length of square ROI in um
int n_regions = 10 // number of regions to generate

int seed = -1
def rng = new Random()
if (seed >= 0)
    rng.setSeed(seed)



selectObjectsByClassification("tissue");
def selected = getSelectedObject()  


tissueAnnotation = getAnnotationObjects().find{it.getPathClass() == getPathClass("tissue")}
tissueroi = tissueAnnotation.getROI()
println(tissueroi.getClass())

def imageData=getCurrentImageData()
pixelwidth = imageData.getServer().getPixelCalibration().getPixelWidth()

println(pixelwidth)
pxsidelength = sidelength/pixelwidth


int count = 0
double x = 0
double y = 0

def xs = [];
def ys = [];

while (count < n_regions) {
    println count
    if (Thread.currentThread().isInterrupted()) {
        println 'Interrupted!'
        return
    }
    x = tissueroi.getBoundsX() + rng.nextDouble() * tissueroi.getBoundsWidth()
    y = tissueroi.getBoundsY() + rng.nextDouble() * tissueroi.getBoundsHeight()

    if (!tissueroi.contains(x + 0.5*pxsidelength, y + 0.5*pxsidelength)) //if the centre of the box is contained in tissueroi, incresase count
        continue
        
    // discard a box if it overlaps with an already existing box 
    def grx = xs.collect{ it - pxsidelength < x }
    def smlx = xs.collect{ it + pxsidelength > x }
    def gry = ys.collect{ it - pxsidelength < y }
    def smly = ys.collect{ it + pxsidelength > y }
    def istrue = [grx, smlx, gry, smly].transpose().collect {it.every{ it }}
    
    if (istrue.any {it})
        continue
    
    xs.add(x);
    ys.add(y);
    count++
    
    // generate ROIs for boxes that passed all conditions
    roi = ROIs.createRectangleROI(x, y, pxsidelength, pxsidelength, ImagePlane.getDefaultPlane())
    pathObject = PathObjects.createAnnotationObject(roi, getPathClass("Region_gap*") )
    addObject(pathObject) 
}

### 2.1.2 Create the training image from annotations 
Run ```Classify ‣ Training images ‣ Create training image```

and select "Other" in the drop-down menu under Classification. Then click "Ok". 

### 2.1.3 Generate training annotations

> Good to know: General tips when generating training annotations  
> * Stay away from the edges of each patch.  
> * Make annotations small and roughly equal size — Many small annotations will better represent the diversity of your data than few large annotations.  
> * Annotations should not overlap  

#### Step by step: 
1. Open the newly generated composite image
2. On each patch derived from each training image, draw a small region that represents the class "gap". Skip a patch if it has no gap in it.  
3. All thus-far made selections therefore belong to the class "gap". Mark all of them in the ```Annotations``` tab.  
(To do so, click "Select all". or shift+click on the first and last annotation. Or cmd+click (ctrl+click on Windows) on each annotation belonging to "gap").
4. With all relevant annotations highlighted, click on the corresponding class in the classification list to the right a click "set selected". All marked annotations should now be assigned to the class "gap".  
5. Repeat steps 2-4, but this time mark regions that are not gaps. Asign these to the class "Ignore*". Be careful not to wrongly re-assign annotations of class "gap" to ignore (and vice versa).  
6. Open ```Classify > Pixel classification > Train Pixel classifier```. Follow [the documentation to train your pixel classifier](https://qupath.readthedocs.io/en/stable/docs/tutorials/pixel_classification.html#getting-started). In short:
7. Below are example settings for the pixel classifier: Only values deviating from standard settings are shown. I recommend experimenting with these settings.  
8. Name your classifier: enter a name in the Classifier name field. Click ```Save```.  
9. Click on ```Live prediction```. This may take some time to load  
10. Inspect the result:  
    * To toggle the prediction on and off, press C on your keyboard.  
    * You can toggle Output from Classification to Probability. Classification shows binary classification, whereas Probability colorcodes the confidence of the prediction.  
    * The accuaracy of the trained pixel classifier can be viewed in the Log. Use shift+cmd(ctrl)+L to open the log. Look for a message that starts with "Current accuracy on the TRAINING SET..." 

11. Once you are happy with the filters, turn off live prediction by clicking on "Live prediction".  
12. Correct wrongly predicted pixels. When output is "Probability", you can search for the most confidently incorrect prediction per patch.  
Repeat point 2-4, except marking regions that were wrongly classified.  
13. Click on "Live prediction" again to re-compute the classification.  
14. Once happy, save your classifier by clicking ```Save```

>A pixelclassifier with these settings can be found in ```pixel_classifiers``` in GitHub named ```gaps_tutorial.json```
>* **Classifier**: Random trees (RTrees)
>* **Resolution**: Very high (0.5 um/px)
>* **Features** > Edit:   
>    - **Channels**:  
>    Hematoxylin,
>    Eosin, 
>    Residual  
>    - **Scales**:  
>    1.0, 
>    4.0, 
>    8.0   
>    - **Features**:  
>    Gaussian, 
>    Gradient magnitude

## 2.2 Apply the classifier to all images

This step may take a long time depending on your image size, compute capability, and classifier settings. Computation on a compute cluster may be necessary.  
You can test it on a single image/small tissue region first.  

Run the following script for one/all images. Modify these variables as necessary:  

**```classifier_name```**: enter the name of the classifier you trained. The classifier should be in ```classifiers/pixel_classifiers```  
**Change this variable to ```"edema_20241006"``` if you want to use the downloaded classifier**   
**```min_area```**: The the minimum area of a region to keep.  Smaller fragments will be discarded.  
**```min_hole_size```**: minimum hole area. Holes smaller than this area will be filled.  
**```name_gaps```**: The name of the QuPath class for gaps between cells. For instance "gaps".  

In [ ]:
classifier_name = "gaps_tutorial" // Enter the name of the classifier you want to use.
min_area = 5.0 // Minimum area (um^2) of fragments to keep 
min_hole_size = 15.0 // minimum area of holes to keep (um^2)
name_gaps = "fluid"

fluid = getAnnotationObjects().findAll{it.getPathClass() == getPathClass(name_gaps)}


if (!fluid) { // only run if fluid is not already annotated
    selectObjectsByClassification("tissue") // Only classify within the tissue annotation
    createAnnotationsFromPixelClassifier(classifier_name, min_area, min_hole_size) 
    fireHierarchyUpdate()
}